In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 15 13:19:13 2019

@author: 60888
"""
from dataInterfaceAssistant import dataInterfaceClass
import pandas as pd
from sklearn import linear_model
import numpy as np
import math
import sys
import datetime

DIA = dataInterfaceClass()

def spongeCityService(start,stop,levelThreshold,maxThreshold,levelSlopeAngle,dataOffset,bufferLength,resultAttribute):
    mille = 1000 # magic number that is used to correct Alibaba UNIX time to standard UNIX time
    timeRange = [start,stop]
    levelThreshold = levelThreshold
    levelSlopeAngle = levelSlopeAngle
    dataOffset = int(dataOffset)
    bufferLength = int(bufferLength)
    
    # Import data 
    # parse json data frame to a panda dataframe using: getDataToWorkWith("data from the API")
    requestedData, statusInfoRequest, reasonCodeRequest = DIA.getDataToWorkWith(timeRange)
    #print(len(requestedData))
    #print(statusInfoRequest)#, reasonCodeRequest)
    #print(requestedData)
    # convert it (if you like) to a panda dataframe for algoritmic convinience: postProcessedDataBack("your converted data")
    requestedDataframe = DIA.jsonToDataframe(requestedData, resultAttribute)
    # Do basic repair
    requestedDataframe.isnull().values.any()
    # mirror baseline input to output and change only what you need to change
    processedDataframe = requestedDataframe
    
    """ ####### Do the data "magic" ########### """ 
    # Remember to add unit test of this particular piece of code - sanity check is only done on data request and data post
    degree = 1
    linearRegressionModel = linear_model.LinearRegression()
  
    for timeIndex in range(0, len(requestedDataframe.get('water_level').values[dataOffset:])-bufferLength, bufferLength):
        
        # Get data to work with
        x = requestedDataframe.get('unix_time').values[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength].reshape(len(requestedDataframe.get('unix_time').values[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength]), 1)
        y = requestedDataframe.get('water_level').values[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength].reshape(len(requestedDataframe.get('water_level').values[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength]), 1)
        x = np.divide(x, mille)
        
        xx = requestedDataframe.get('unix_time').values[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength].astype('int')
        yy = requestedDataframe.get('water_level').values[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength].astype('float')
        xx = np.divide(xx, mille)
        
        # Make line fit / linear regression
        weights = np.polyfit(np.array(xx),np.array(yy), degree)
        model = np.poly1d(weights)
        
        #linearRegressionModel.fit(x,y)
        #linearRegressionModelPrediction = linearRegressionModel.predict(list(test.get('time').values))
        
        #bufferSlope = math.degrees(math.atan(linearRegressionModel.coef_))
        bufferSlope = weights[0]
        #bufferMean = np.mean(y)
        bufferMean = np.mean(yy)

        # Make primitive overflow classification
        if bufferMean >= levelThreshold and bufferSlope > levelSlopeAngle:
            #print('yes')
            processedDataframe.loc[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength-1,resultAttribute] = requestedDataframe.get('water_level').values[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength]
        elif bufferMean >= maxThreshold and bufferSlope >= 0:
            #print('yes')
            processedDataframe.loc[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength-1,resultAttribute] = requestedDataframe.get('water_level').values[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength]
        else:
            #print('no')
            processedDataframe.loc[dataOffset+timeIndex:dataOffset+timeIndex+bufferLength-1,resultAttribute] = float(-1)
            
    """ ####### Please stop your "magic" ########### """    
    # Export data
    processedData = DIA.dataframeToJson(processedDataframe)
    #print(processedData)
    # ship it away with: postProcessedDataBack("your converted data")
    statusInfoPost, reasonCodePost = DIA.postProcessedDataBack(processedData)
    #print(statusInfoPost)#, reasonCode)
    if(statusInfoRequest == DIA.sanityCheck() and statusInfoPost == DIA.sanityCheck() and len(requestedData) > 0):
        sanityCheck = True
    else:
        sanityCheck = False
            
    print("The operation went well:", sanityCheck)
    return processedDataframe, sanityCheck
        
if __name__ == '__main__':
    timeOffset = 72 # Number of hours to take into account in the overflow post process
    localDate = datetime.datetime.now() - datetime.timedelta(hours = timeOffset)
    localDateStart = localDate.strftime("%Y-%m-%dT%H:%M:%SZ")
    localDatePrevious = localDate + datetime.timedelta(hours = timeOffset)
    localDateStop = localDatePrevious.strftime("%Y-%m-%dT%H:%M:%SZ")
    #utcDate = datetime.datetime.utcnow();
    start = '2019-06-04T0:55:52Z' # fixed time stamp used for debugging
    #start = localDateStart
    #print(start)
    stop = '2019-06-05T19:28:52Z' # fixed time stamp used for debugging
    #stop = localDateStop
    dataOffset = 0 # handle bad data engineering work - if you trust your data engineer set it to: 0
    bufferLength = 30 # number of samples per overflow iteration evalution
    levelThreshold = 0.45 # meters of water in pump pit
    maxThreshold = 0.90 # meters of water in pump pit
    levelSlopeAngle = 0.000085 # water increase factor in pump pit
    resultAttribute = 'overflow' # explictly declare your result attribute
    data, test = spongeCityService(start,stop,levelThreshold, maxThreshold, levelSlopeAngle,dataOffset,bufferLength,resultAttribute)
    
    # For Cloud command-line execution
    #print(sys.argv[1:])
    #pumpPitOverflowService(*sys.argv[1:])